# Notebook for data visualization of the processed HFO data from human SEEGs

### Check WD (change if necessary) and file loading

In [58]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/seeg_data/synthetic" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/seeg_data/synthetic"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/seeg_data/synthetic


## Add the parent directory to the path to detect the utils module

In [59]:
import os
import sys

# Add the parent directory to the path so it detects the utils module
module_path = os.path.abspath(os.path.join('src'))      # Changed this since WD is not the same as the file location
if module_path not in sys.path:
    sys.path.append(module_path)

## Load the data from the .npy file

In [60]:
import numpy as np
import math

seeg_file_name = "seeg_synthetic_humans.npy"
recorded_data = np.load(f"{PATH_TO_FILE}{seeg_file_name}")

print("Data shape: ", recorded_data.shape)
print("First time steps: ", recorded_data[:10])

Data shape:  (245760, 960)
First time steps:  [[ 3.2352024e-01 -1.3235390e+00 -5.9668809e-01 ... -1.9608999e+00
  -1.9769822e-01 -1.2078454e+00]
 [-6.9759099e-04 -3.5122361e+00 -4.8766956e-01 ... -5.8757830e+00
  -7.4400985e-01 -5.1096064e-01]
 [ 1.9026639e+00 -5.6726017e+00  9.8274893e-01 ... -6.6182971e+00
  -8.3053267e-01 -8.1596655e-01]
 ...
 [ 3.2172418e+00 -8.4650068e+00  1.5216088e+00 ... -4.1081657e+00
   2.0085973e-01 -4.7539668e+00]
 [ 1.7725919e+00 -9.4744024e+00  1.6776791e+00 ... -4.1469693e+00
   1.6412770e+00 -3.4672713e+00]
 [ 7.8109097e-01 -1.0500931e+01  2.3717029e+00 ... -5.1762242e+00
   1.0715837e+00 -4.4489903e+00]]


## Markers' Shape
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [61]:
markers_seeg_file_name = "seeg_synthetic_humans_markers.npy"
markers = np.load(f"{PATH_TO_FILE}{markers_seeg_file_name}")

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (960, 42)
First time steps:  [[('Spike+Ripple+Fast-Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   4537.6 , 0.)
  ('Ripple+Fast-Ripple',   7610.84, 0.) ('Spike',  10261.2 , 0.)
  ('Fast-Ripple',  13411.6 , 0.) ('Spike+Ripple',  15644.  , 0.)
  ('Ripple',  18607.9 , 0.) ('Fast-Ripple',  21300.8 , 0.)
  ('Fast-Ripple',  24086.9 , 0.)
  ('Spike+Ripple+Fast-Ripple',  26598.1 , 0.)
  ('Spike+Fast-Ripple',  30529.8 , 0.) ('Ripple',  33387.7 , 0.)
  ('Spike+Fast-Ripple',  36145.5 , 0.) ('Spike',  38744.1 , 0.)
  ('Spike+Fast-Ripple',  41876.  , 0.)
  ('Ripple+Fast-Ripple',  44531.7 , 0.) ('Spike+Ripple',  47699.7 , 0.)
  ('Spike+Ripple+Fast-Ripple',  50303.2 , 0.) ('Ripple',  53045.9 , 0.)
  ('Ripple+Fast-Ripple',  56416.  , 0.)
  ('Spike+Ripple+Fast-Ripple',  58838.4 , 0.)
  ('Ripple+Fast-Ripple',  62578.6 , 0.) ('Fast-Ripple',  65430.2 , 0.)
  ('Spike',  68184.1 , 0.) ('Spike+Ripple',  70753.9 , 0.)
  ('Ripple+Fast-Ripple',  73788.6 , 0.) ('Spike',  76749.  , 0.)
  

## Define some important parameters of the input data

In [62]:
sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds
num_samples = recorded_data.shape[0]    # 2048 * 120 = 245760
num_channels = recorded_data.shape[1]   # 960

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

## Plot the data in an interactive Line Plot

### Configurable parameters

In [63]:
# Channels to plot
min_channel = 0     #   Index of the first channel to plot
max_channel = 10    # num_channels     #   Index of the last channel to plot

# Create set containing the indices of the channels used
channels_used = {0, 60, 120, 180, 240, 360, 480, 600, 660, 720, 780, 840}    # {94}  #  # {0, 1, 30, 31, 60, 61, 90, 91} 
# channels_used = set(range(1, max_channel+1, 1))

# Sort the set by value and convert it to a list
channels_used = sorted(channels_used)

print(channels_used)

[0, 60, 120, 180, 240, 360, 480, 600, 660, 720, 780, 840]


In [64]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

# Create the y arrays for the voltage plot representing the voltage of each electrode
v_yarrays = []
# Add each channel
for ch_idx in channels_used:
    v_yarrays.append([val[ch_idx] for val in recorded_data])

## Define a specific time interval to plot

In [65]:
min_t = 0
max_t = input_duration # / 10     # 120 / 10 seconds = 12 seconds

# Trim the x and y arrays to the desired time range
start_index = int(min_t / x_step)
num_data_points = int((max_t - min_t) / x_step)

x = x[start_index:start_index + num_data_points + 1]
for i in range(len(v_yarrays)):
    v_yarrays[i] = v_yarrays[i][start_index:start_index + num_data_points + 1]  # Trim the y arrays

print("num_data_points", num_data_points)

num_data_points 245760


## Create the Plot

In [66]:
# Create the plot
# List of tuples containing the y values and the legend label
hfo_y_arrays = [(voltage_val, f"Ch. {idx}") for idx, voltage_val in enumerate(v_yarrays)]

# Create the SEEG Voltage plot
hfo_plot = create_fig(
    title="SEEG Voltage dynamics of Synthetic data from Human recordings", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (μV)',
    x=x, 
    y_arrays=hfo_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    # y_range=Range1d(-0.05, 1.05)
)

# If there are more than 30 channels, hide the legend
if len(channels_used) > 30:
    # Hide the legend
    hfo_plot.legend.visible = False

## Add Box Annotations to the plot to identify the marked HFOs (ground truth)

In [67]:
from bokeh.models import BoxAnnotation

confidence_range = 100          # When the duration is missing (0), we consider the 200ms window around the marked position
color_map = {                   # Map the label to a color
    'Spike': 'red',
    'Fast-Ripple': 'blue',
    'Ripple': 'green',  
    'Spike+Ripple': 'yellow',
    'Spike+Fast-Ripple': 'magenta',
    'Ripple+Fast-Ripple': 'cyan',
    'Spike+Ripple+Fast-Ripple': 'black'
}

visited_markers = {}    # Avoid inserting multiple boxes for the same marker (only one of each label)
use_visited = False     # Boolean controlling if we remove duplicate markers

plot_instant = True     # Boolean to plot the markers as instant events or as boxes
instant_width = 100 # 20       # Width of the instant event for visualization purposes

show_markers = False    # Boolean to show the markers
if show_markers:
    for ch_idx in channels_used:
        channel_markers = markers[ch_idx]
        # print("channel_markers", channel_markers)
        for idx2, marker in enumerate(channel_markers):
            # print("marker:", marker)
            
            if use_visited:
                # Check if the marker has already been visited and skip it if it has
                if marker['position'] in visited_markers:
                    visited_labels = visited_markers[marker['position']]    # Get the labels that already have an annotation for this position
                    if marker['label'] in visited_labels:
                        # print("Skipping marker", marker['position'], marker['label'])
                        continue    # Skip this marker
                    else:
                        visited_labels.append(marker['label'])  # Add the label to the visited labels
                else:
                    visited_markers[marker['position']] = [marker['label']] # Add the marker to the visited markers

            # Add a box annotation for each marker
            has_duration = marker['duration'] > 0
            
            confidence_constant = 0 if plot_instant or has_duration else confidence_range
            instant_padding = instant_width if plot_instant else 0

            left = marker['position'] - confidence_constant
            right = marker['position'] + confidence_constant + instant_width
            box_color = color_map[marker['label']]  # Choose a color according to the label
            
            if left < min_t or right > max_t:
                continue    # Skip this marker
            

            box = BoxAnnotation(left=left, right=right, fill_color=box_color, fill_alpha=0.1)
            # print("Added marker for channel: ", ch_idx, " at position: ", left)
            hfo_plot.add_layout(box)

## Show the Plot

In [68]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    bplt.show(hfo_plot)

KeyboardInterrupt: 

## Export the plot to a file

In [ ]:
export = True
FILENAME = f"seeg_all_regions_diff_snr"     # f"seeg_ch{min_channel}_{max_channel}"    

if export:
    file_path = f"{PATH_TO_FILE}results/{FILENAME}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="SEEG Data - Voltage dynamics across time")

    # Save the plot
    bplt.save(hfo_plot)

In [ ]:
# TODO: Analyze the HFOs of different channels unde